## Project Overview  
Analyze the contact angle, and water conformation on TiO2 slab in three different force fields :nonbonded and bonded by Lyubartsev and Machesky.  

(Actually there are only two forcefields after double-checking the papers.)

## Simulation to Do  
Contact angles will be computed using MD simulation of 200 water molecules in the NVT ensemble at 298.15 K on TiO2 surfaces. Two different force fields will be employed. The water molecules will be initially placed in a cylindrical configuration. The dimensions of the surfaces are ~5×5 nm2 to allow spreading of water molecules and avoid interactions with periodic images. The total simulation time are 3 ns, of which the first 1 ns serves for equilibration and the last 2 ns are used to collect data. The obtained trajectories of superficial water will be visualized using the VMD graphical interface and superimposed to obtain a statistically averaged geometry of the water droplets formed. Contact angles will be graphically analyzed from rendered images.

## Generate Cylindrical Water  
* First, I generate a box of water using "gmx solvate" to specify -cs spc216.gro and -box (5 5 5). After getting a cubic water box with radius=5nm, I put it into VMD to cut out a circular piece in the center of xy-plane and check the number of water molecules is around 200.  

<img src="pictures/water_cubic.png">

<img src="pictures/water_cylinder.png">

Also, we can use packmol to generate a cylinder conveniently:  

inside cylinder  a1  b1  c1  a2  b2  c2  d  l 

(a1 b1 c1) define the point where the cylinder starts. The orientation of the cylinder, parallel to which line is defined by the director vector (a2 b2 c2), the fourth, fifth and sixth parameters. And l defines the length of the cylinder. d defines de radius of the cylinder.  

inside cylinder 0. 0. 0. 1. 0. 0. 10. 20. 


## Merge the Surface and Water  
* First, I tried to put the water cylinder close to the surface and merged these two files using the command Kayla told us^^. So we got a new file called merged.pdb. Note that don't forget to modify the topol.top file to include the water molecules in [ molecules ] in the same order as that in merged.pdb.  

<img src="pictures/merged.png">

<img src="pictures/topol.png">

## Run the Simulation

### Do the energy minimization.

#### 1. Error I got:

After em, I got the state like this:

<img src="pictures/em_nofreezing.png">

#### 2. Solutions I tried:

- In light of the corruption of the surface layer, I tried freezing the surface in em.mdp file.  

freezegrps     = TIO  
freezedim      = Y Y Y

### Do the NVT.

I ran mdrun to simulate the system in NVT ensemble (since our simulation should be under vacuum conditions). However, I made a mistake here and I tried to find out and solve it as follows:

#### 1. Error I got:

while NVT simulation, it was interrupted and showed error below:

<img src="pictures/LINCS_warnings.png">

and I googled and asked Coco and Wesley to find out that it was probably because ** the system energy was too high**.


#### 2. Solutions I tried:

- I first tried to move the water somehow away from the surface twice to see if the whole energy would be lower.  
Failed :(


- Then I tried to do the em for either water cube or water cylinder first and then merged the system.  
Failed again :(


- At last! Finally! I tried to visualize the em.gro file to check the status of the whole system after em. Surprisingly, the water got through the surface like this:

<img src="pictures/error_em.png">

    And I used "pbc box" in vmd to see if my box was not big enough for the system.
    
    And I checked my merged.pdb file to find out that I didn't change the box size (especially the height) after combining the system. That's the problem.

** Note: It's different from the tutorial or what we have done before, which is, defining a box and filling it with solvent where the box size would remain.**

### Do the NVT simulation. 

Change the nsteps in nvt.mdp file to set the whole simulation time as 3ns. When it's done, I used trjconv to account for any periodicity in the system.  
（The protein will diffuse through the unit cell, and may appear to "jump" across to the other side of the box.）

gmx trjconv -s nvt.tpr -f nvt.xtc -o nvt_noPBC.xtc -pbc mol -ur compact

### Contact Angle.

** Both are 0 for bonded and nonbonded forcefields!**

I got an ideal representation of nonbonded forcefield, where the water just quickly spread out on the surface of TIO2 within the first 1ns. However, some weird phenomena were observed of bonded forcefield.

** Bonded Forcefield**

<img src="pictures/bonded_per.png">

Then, I tried to rise the height of the box from 7nm to 10nm and that worked!

Buttt I'm still not pretty sure why LOL..

<img src="pictures/bonded_done.png">

** Nonbonded Forcefield**

<img src="pictures/nbonded_done.png">

### Another Question about # Frames

Like, I got 77 frames at last in vmd and I wanted to generated more frames to show the whole simulation process more continuously. So I tried to shrink my "nstxout" (number of steps that elapse between writing coordinates to output trajectory file) and "nstxtcout" (frequency to write coordinates to xtc trajectory?) from 10000 to 100/from 20000 to 200 respectively. However the number of frames remains the same. 

<img src="pictures/output freq.png">

Then I tried to reduce a hundredfold of "nstlog" and "nstenergy" simultaneously. It worked! (Why?) But the number of frames seems just doubled or tripled actually..  

Q: How to control # Frames accurately? 

## Future work  

1. Even though both of our contact angles are zero, I want to find out if they're not, how to calculate them theoretically.  

    Possible Methods: "The contact angle was determined by computing water density maps in the plane of the center of mass of the drop. The curve with a density equal to half of Fb was then fit to a circle, and the angle between the tangent to this circle at zS = 7 Å and the horizontal axis was taken to be the contact angle."  

    *Molecular Explanation for Why Talc Surfaces Can Be Both Hydrophilic and Hydrophobic Benjamin Rotenberg,*,† Amish J. Patel,‡ and David Chandler§*  

2. Find some experimental results from papers to verify my simulation.

3. Make a movie of the trajectories.